In [ ]:
from app.data.utils.bigquery import bigquery
from app.data.utils.load_query import load_format
from app.conf.settings import DEFAULT_PARAMS

In [ ]:
sql = '''
DECLARE SITE_ID STRING DEFAULT "{site_id}";
DECLARE START_DATE DATE DEFAULT "2023-01-23";

SELECT
    COUNT(*)
FROM
    meli-bi-data.MELIDATA.ADVERTISING
WHERE
    `ds` >= START_DATE
    AND `event` = 'display_prints'
    AND site = SITE_ID
    AND json_extract_scalar(`event_data`, '$.content_source') = 'DSP'
    AND json_extract_scalar(event_data, '$.valid')= 'true'
    AND (NOT
        REGEXP_CONTAINS(LOWER(`device`.user_agent),
        '.*(libwww|wget|lwp|damnBot|bbbike|java|spider|crawl|slurp|bot|feedburner|googleimageproxy|google web preview).*')
    )
LIMIT 1000
'''

params = {'site_id': 'MLB'}
sql = sql.format(**params)
df = bigquery.run_query(sql)
df

In [ ]:
filename = '../src/app/data/creatives/queries/historic_daily_performance.sql'
sql = load_format(filename, DEFAULT_PARAMS)
bq.run_query(sql)

In [ ]:
sql ='''
DECLARE SITE_ID STRING DEFAULT '{site_id}';
CREATE OR REPLACE TABLE meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_LAST_DATE_HOUR AS
SELECT SITE_ID AS site, MAX(CONCAT(ds, 'T', hour)) AS ds_hour
FROM meli-bi-data.SBOX_DSPCREATIVOS.BQ_PRINTS_CLICKS_PER_HOUR
WHERE site = SITE_ID
'''
sql = sql.format(**DEFAULT_PARAMS)
bq.run_query(sql)

In [ ]:
filename = '../src/app/data/creatives/queries/daily_insert.sql'
sql = load_format(filename, DEFAULT_PARAMS)
bq.run_query(sql)

In [ ]:
filename = '../src/app/data/creatives/queries/group.sql'
sql = load_format(filename, DEFAULT_PARAMS)
bq.run_query(sql)

In [ ]:
sql ='''
DECLARE SITE_ID STRING DEFAULT '{site_id}';
CREATE OR REPLACE TABLE meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_LAST_DATE_HOUR AS
SELECT DISTINCT * FROM meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_LAST_DATE_HOUR
WHERE site = SITE_ID
'''
sql = sql.format(**DEFAULT_PARAMS)
bq.run_query(sql)

In [ ]:
sql ='''
DECLARE SITE_ID STRING DEFAULT '{site_id}';
DECLARE max_ds_hour STRING;

SET max_ds_hour = (SELECT MAX(max_ds_hour) FROM meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_LAST_DATE_HOUR WHERE site = SITE_ID);

DELETE FROM meli-bi-data.SBOX_DSPCREATIVOS.BETA_ESTIMATION_LAST_DATE_HOUR
WHERE site = SITE_ID AND ds_hour = max_ds_hour;
'''
sql = sql.format(**DEFAULT_PARAMS)
bq.run_query(sql)